[old version of findmax(f, domain)](
https://github.com/JuliaLang/julia/pull/35316/files#diff-97ad8b900c30b39398c32d78754c404f4a6df9a153d45284b2c11ab54837deb9R797)

```julia
findmax(f, domain) = mapfoldl(x -> (f(x), x), _rf_findmax, domain)
 _rf_findmax((fm, m), (fx, x)) = isless(fm, fx) ? (fx, x) : (fm, m)
```

[current version](https://github.com/JuliaLang/julia/pull/41076/files#diff-97ad8b900c30b39398c32d78754c404f4a6df9a153d45284b2c11ab54837deb9R803)

```julia
findmax(f, domain) = mapfoldl( ((k, v),) -> (f(v), k), _rf_findmax, pairs(domain) )
 _rf_findmax((fm, im), (fx, ix)) = isless(fm, fx) ? (fx, ix) : (fm, im)
```

In [1]:
VERSION

v"1.7.0-beta2"

In [2]:
f(x, y) = cos(x)*sin(y) + 0.1(x - y)

X = range(-2, 2; length=401)
Y = range(-2, 2; length=401)
XtimesY = Iterators.product(X, Y)

Base.Iterators.ProductIterator{Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}}((-2.0:0.01:2.0, -2.0:0.01:2.0))

In [3]:
val, idx = findmax(XtimesY) do (x, y); f(x, y) end

LoadError: MethodError: no method matching keys(::Base.Iterators.ProductIterator{Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}})
[0mClosest candidates are:
[0m  keys([91m::IOContext[39m) at show.jl:345
[0m  keys([91m::Base.Generator[39m) at generator.jl:54
[0m  keys([91m::Tuple[39m) at tuple.jl:72
[0m  ...

`pairs` function used in `findmax(f, domain)` function requires `keys` method.

In [4]:
methods(keys)

# 20 methods for generic function "keys":
[1] keys(io::IOContext) in Base at show.jl:345
[2] keys(g::Base.Generator) in Base at generator.jl:54
[3] keys(t::Tuple) in Base at tuple.jl:72
[4] keys(t::Tuple, t2::Tuple...) in Base at tuple.jl:77
[5] keys(itr::Base.SkipMissing) in Base at missing.jl:264
[6] keys(::Number) in Base at number.jl:93
[7] keys(v::Base.Pairs) in Base.Iterators at iterators.jl:256
[8] keys(a::AbstractDict) in Base at abstractdict.jl:103
[9] keys(nt::NamedTuple{names}) where names in Base at namedtuple.jl:306
[10] keys(s::IndexStyle, A::AbstractArray, B::AbstractArray...) in Base at abstractarray.jl:353
[11] keys(a::Test.GenericArray) in Test at D:\Julia-1.7.0\share\julia\stdlib\v1.7\Test\src\Test.jl:1819
[12] keys(a::AbstractVector) in Base at abstractarray.jl:136
[13] keys(a::AbstractArray) in Base at abstractarray.jl:135
[14] keys(s::AbstractString) in Base at strings/basic.jl:571
[15] keys(io::IO) in Base at show.jl:346
[16] keys(cmd::Cmd) in Base at process.jl:643
[17] keys(m::RegexMatch) in Base at regex.jl:219
[18] keys(m::Pkg.Types.Manifest) in Pkg.Types at D:\Julia-1.7.0\share\julia\stdlib\v1.7\Pkg\src\Types.jl:279
[19] keys(v::Core.SimpleVector) in Base at essentials.jl:609
[20] keys(r::Pkg.Registry.RegistryInstance) in Pkg.Registry at D:\Julia-1.7.0\share\julia\stdlib\v1.7\Pkg\src\Registry\registry_instance.jl:386

In [5]:
keys(collect(XtimesY))

401×401 CartesianIndices{2, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}}}:
 CartesianIndex(1, 1)    CartesianIndex(1, 2)    …  CartesianIndex(1, 401)
 CartesianIndex(2, 1)    CartesianIndex(2, 2)       CartesianIndex(2, 401)
 CartesianIndex(3, 1)    CartesianIndex(3, 2)       CartesianIndex(3, 401)
 CartesianIndex(4, 1)    CartesianIndex(4, 2)       CartesianIndex(4, 401)
 CartesianIndex(5, 1)    CartesianIndex(5, 2)       CartesianIndex(5, 401)
 CartesianIndex(6, 1)    CartesianIndex(6, 2)    …  CartesianIndex(6, 401)
 CartesianIndex(7, 1)    CartesianIndex(7, 2)       CartesianIndex(7, 401)
 CartesianIndex(8, 1)    CartesianIndex(8, 2)       CartesianIndex(8, 401)
 CartesianIndex(9, 1)    CartesianIndex(9, 2)       CartesianIndex(9, 401)
 CartesianIndex(10, 1)   CartesianIndex(10, 2)      CartesianIndex(10, 401)
 CartesianIndex(11, 1)   CartesianIndex(11, 2)   …  CartesianIndex(11, 401)
 CartesianIndex(12, 1)   CartesianIndex(12, 2)      CartesianIndex(12, 401)
 CartesianIndex(13, 1) 

In [6]:
val, idx = findmax(collect(XtimesY)) do (x, y); f(x, y) end

(0.8529538721652244, CartesianIndex(211, 348))

But `collect` causes memory allocations.

In [7]:
@time collect(XtimesY);

  0.000737 seconds (2 allocations: 2.454 MiB)


In [8]:
axes(XtimesY)

(Base.OneTo(401), Base.OneTo(401))

In [9]:
CartesianIndices(axes(XtimesY))

401×401 CartesianIndices{2, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}}}:
 CartesianIndex(1, 1)    CartesianIndex(1, 2)    …  CartesianIndex(1, 401)
 CartesianIndex(2, 1)    CartesianIndex(2, 2)       CartesianIndex(2, 401)
 CartesianIndex(3, 1)    CartesianIndex(3, 2)       CartesianIndex(3, 401)
 CartesianIndex(4, 1)    CartesianIndex(4, 2)       CartesianIndex(4, 401)
 CartesianIndex(5, 1)    CartesianIndex(5, 2)       CartesianIndex(5, 401)
 CartesianIndex(6, 1)    CartesianIndex(6, 2)    …  CartesianIndex(6, 401)
 CartesianIndex(7, 1)    CartesianIndex(7, 2)       CartesianIndex(7, 401)
 CartesianIndex(8, 1)    CartesianIndex(8, 2)       CartesianIndex(8, 401)
 CartesianIndex(9, 1)    CartesianIndex(9, 2)       CartesianIndex(9, 401)
 CartesianIndex(10, 1)   CartesianIndex(10, 2)      CartesianIndex(10, 401)
 CartesianIndex(11, 1)   CartesianIndex(11, 2)   …  CartesianIndex(11, 401)
 CartesianIndex(12, 1)   CartesianIndex(12, 2)      CartesianIndex(12, 401)
 CartesianIndex(13, 1) 

In [10]:
Base.keys(pr::Iterators.ProductIterator) = CartesianIndices(axes(pr))
val, idx = findmax(XtimesY) do (x, y); f(x, y) end

(0.8529538721652244, CartesianIndex(211, 348))

In [11]:
keys(XtimesY)
@time keys(XtimesY);

  0.000004 seconds (1 allocation: 32 bytes)


In [12]:
X[idx[1]], Y[idx[2]]

(0.1, 1.47)

In [13]:
pt = argmax(XtimesY) do (x, y); f(x, y) end

(0.1, 1.47)

In [14]:
valargmax(f, X) = (x = argmax(f, X); (f(x), x))
valargmax(XtimesY) do (x, y); f(x, y) end

(0.8529538721652244, (0.1, 1.47))

In [15]:
valargmax(X) = valargmax(Base.Fix1(getindex, X), keys(X))
valargmax((pt -> f(pt...)).(XtimesY))

(0.8529538721652244, CartesianIndex(211, 348))

In [16]:
argmax((pt -> f(pt...)).(XtimesY))

CartesianIndex(211, 348)

In [17]:
findmax(pairs(X)) do (i, x); sin(x) end

LoadError: BoundsError: attempt to access Float64 at index [2]

In [18]:
findmax(sin, pairs(X))

(0.9999996829318346, 358)

In [19]:
pairs(pairs(X)) == pairs(X)

true

In [20]:
@which findmax(sin, pairs(X))

findmax(f, domain) in Base at reduce.jl:803

In [21]:
@which pairs(pairs(X))

pairs(a::AbstractDict) in Base at abstractdict.jl:140

https://github.com/JuliaLang/julia/blob/b570546b68de16bd208ca76a20c1385919de18d6/base/iterators.jl#L236

```julia
# pairs(v::Pairs) = v # listed for reference, but already defined from being an AbstractDict
```

In [22]:
?Base.Pairs

```
Iterators.Pairs(values, keys) <: AbstractDict{eltype(keys), eltype(values)}
```

Transforms an indexable container into a Dictionary-view of the same data. Modifying the key-space of the underlying data may invalidate this object.


https://github.com/JuliaLang/julia/blob/87e08d94a99ea17e7f72493947595eb6b63e0f09/base/essentials.jl#L32

In [23]:
Base.keys(rv::Iterators.Reverse) = keys(reverse(rv.itr))
findmax(sin, Iterators.Reverse(X))

(0.9999996829318346, 44)

In [24]:
Base.keys(en::Iterators.Enumerate) = keys(en.itr)
findmax(Iterators.enumerate(X)) do (i, x); sin(x) end

(0.9999996829318346, 358)

In [25]:
Base.keys(zp::Iterators.Zip) = Base.OneTo(length(zp))
findmax(Iterators.zip(X, reverse(Y))) do (x, y); cos(x)*sin(y) end

(0.49997882324937, 122)

In [26]:
Base.keys(ac::Iterators.Accumulate) = keys(ac.itr)
findmax(sin, Iterators.accumulate(+, X; init=π))

(0.9999998945903477, 170)

In [27]:
Base.keys(tk::Iterators.Take) = Base.OneTo(tk.n)
findmax(sin, Iterators.take(reverse(X), 100))

(0.9999996829318346, 44)

In [28]:
methods(keys, Main)

# 6 methods for generic function "keys":
[1] keys(pr::Base.Iterators.ProductIterator) in Main at In[10]:1
[2] keys(en::Base.Iterators.Enumerate) in Main at In[24]:1
[3] keys(rv::Base.Iterators.Reverse) in Main at In[23]:1
[4] keys(tk::Base.Iterators.Take) in Main at In[27]:1
[5] keys(ac::Base.Iterators.Accumulate) in Main at In[26]:1
[6] keys(zp::Base.Iterators.Zip) in Main at In[25]:1

In [29]:
?keys

search: keys keytype KeyError haskey getkey UndefKeywordError WeakKeyDict



```
keys(a::AbstractArray)
```

Return an efficient array describing all valid indices for `a` arranged in the shape of `a` itself.

They keys of 1-dimensional arrays (vectors) are integers, whereas all other N-dimensional arrays use [`CartesianIndex`](@ref) to describe their locations.  Often the special array types [`LinearIndices`](@ref) and [`CartesianIndices`](@ref) are used to efficiently represent these arrays of integers and `CartesianIndex`es, respectively.

Note that the `keys` of an array might not be the most efficient index type; for maximum performance use  [`eachindex`](@ref) instead.

---

```
keys(iterator)
```

For an iterator or collection that has keys and values (e.g. arrays and dictionaries), return an iterator over the keys.

---

```
keys(a::AbstractDict)
```

Return an iterator over all keys in a dictionary. `collect(keys(a))` returns an array of keys. When the keys are stored internally in a hash table, as is the case for `Dict`, the order in which they are returned may vary. But `keys(a)` and `values(a)` both iterate `a` and return the elements in the same order.

# Examples

```jldoctest
julia> D = Dict('a'=>2, 'b'=>3)
Dict{Char, Int64} with 2 entries:
  'a' => 2
  'b' => 3

julia> collect(keys(D))
2-element Vector{Char}:
 'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
 'b': ASCII/Unicode U+0062 (category Ll: Letter, lowercase)
```

---

```
keys(m::RegexMatch) -> Vector
```

Return a vector of keys for all capture groups of the underlying regex. A key is included even if the capture group fails to match. That is, `idx` will be in the return value even if `m[idx] == nothing`.

Unnamed capture groups will have integer keys corresponding to their index. Named capture groups will have string keys.

!!! compat "Julia 1.6"
    This method was added in Julia 1.6


# Examples

```jldoctest
julia> keys(match(r"(?<hour>\d+):(?<minute>\d+)(am|pm)?", "11:30"))
3-element Vector{Any}:
  "hour"
  "minute"
 3
```


https://github.com/JuliaLang/julia/blob/16f433bb13cfc87eea21d26a797dac7b34a41d86/base/abstractdict.jl#L71